<h2>3D stack - Batch Processing - Marker+ based on APOC Object Classifier</h2>

The following notebook is able to process a 3D stack (.czi or .nd2 files) and allows you to:

1. Inspect your images in Napari.
2. Define regions of interest (ROIs) using labels in Napari. Store said ROIs as .tiff files if needed.
3. Predict nuclei labels and store them as .tiff files for further processing.
4. Extract numbers of cells positive for a marker based on pre-trained object classifiers (scikit random forest).
5. Display positive cells in Napari.
6. Extract and save number of positive cells in a .csv file (SP_marker_+_label_obj_class).

In [22]:
from pathlib import Path
import tifffile
import napari
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import pyclesperanto_prototype as cle
import apoc
from skimage import measure
from utils_stardist import get_gpu_details, list_images, read_image, extract_nuclei_stack, get_stardist_model, maximum_intensity_projection, simulate_cytoplasm_chunked_3d, simulate_cell_chunked_3d, simulate_cytoplasm, simulate_cell, segment_nuclei, remove_labels_touching_roi_edge

get_gpu_details()
cle.select_device("RTX")

Device name: /device:GPU:0
Device type: GPU
GPU model: device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


<NVIDIA GeForce RTX 4090 on Platform: NVIDIA CUDA (2 refs)>

<h3>Define the directory where your images are stored (.nd2 or .czi files)</h3>

In [23]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("../raw_data/test_data")

# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr, cellular_location),(..., ..., ...)]
# cellular locations can be "nucleus", "cytoplasm" or "cell" (cell being the sum volume of nucleus and cytoplasm)
# Remember in Python one starts counting from 0, so your first channel will be 0
# i.e. markers = [("ki67", 0, "nucleus"), ("neun", 1, "cell"), ("calbindin", 2, "cytoplasm")]
markers = [("ki67", 0, "nucleus"), ("neun", 1, "cell"), ("calbindin", 2, "cytoplasm")]

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

images

['..\\raw_data\\test_data\\HI1_CONTRA_M8_S6_TR1.czi',
 '..\\raw_data\\test_data\\HI1_CONTRA_M8_S6_TR2.czi',
 '..\\raw_data\\test_data\\HI1_CONTRA_M8_S7_TR1.czi',
 '..\\raw_data\\test_data\\HI1_CONTRA_M8_S7_TR2.czi',
 '..\\raw_data\\test_data\\HI1_IPSI_M8_S6_TR1.czi',
 '..\\raw_data\\test_data\\HI1_IPSI_M8_S6_TR2.czi',
 '..\\raw_data\\test_data\\HI1_IPSI_M8_S7_TR1.czi',
 '..\\raw_data\\test_data\\HI2_CONTRA_M10_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI2_CONTRA_M10_S10_TR2.czi',
 '..\\raw_data\\test_data\\HI2_IPSI_M10_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI2_IPSI_M10_S10_TR2.czi',
 '..\\raw_data\\test_data\\HI3_CONTRA_M11_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI3_CONTRA_M11_S10_TR2.czi',
 '..\\raw_data\\test_data\\HI3_IPSI_M11_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI3_IPSI_M11_S10_TR2.czi',
 '..\\raw_data\\test_data\\SHAM1_CONTRA_M6_S11_TR1.czi',
 '..\\raw_data\\test_data\\SHAM1_CONTRA_M6_S11_TR2.czi',
 '..\\raw_data\\test_data\\SHAM1_CONTRA_M7_S11_TR1.czi',
 '..\\raw_data\\t

<h3>Open each image in the directory</h3>
You can do so by changing the number within the brackets below <code>image = images[0]</code>. By changing the <code>slicing factor</code> you lose resolution but speed up processing times (check the results).

If you have not generated nuclei predictions before, input <code>nuclei_channel</code>, <code>n_tiles</code>, <code>segmentation_type</code> and <code>model_name</code> values.

In [24]:
# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
# Now, in addition to xy, you can downsample across your z-stack
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)
slicing_factor_z = None # Use 2 to select 1 out of every 2 z-slices

# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 3

# The n_tiles parameter defines the number of tiles the input volume/image will be divided into along each dimension (z, y, x) during prediction. 
# This is useful for processing large images that may not fit into memory at once.
# While tiling can handle memory limitations, chopping the image into smaller chunks increases
# the processing time for stitching the predictions back together. 
# Use n_tiles=(1, 1, 1) if the input volume fits in memory without tiling to minimize processing overhead.
n_tiles=(1,4,4)

# Segmentation type ("2D" or "3D"). 
# 2D takes a z-stack as input, performs MIP (Maximum Intensity Projection) and predicts nuclei from the resulting projection (faster, useful for single layers of cells)
# 3D is more computationally expensive. Predicts 3D nuclear volumes, useful for multilayered structures
segmentation_type = "3D"

# Nuclear segmentation model type ("Stardist")
# Choose your Stardist fine-tuned model (model_name) from stardist_models folder
# If no custom model is present, type "test" and a standard pre-trained model will be loaded
model_name = "MEC0.1" # Type "test" if you don't have a custom model trained

# Model loading 
model = get_stardist_model(segmentation_type, name=model_name, basedir='stardist_models')

Loading 3D segmentation model
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.583933, nms_thresh=0.3.


<h3>Mask the input image with the user defined ROIs, apply object classifiers and extract data</h3>

In [25]:
# Construct ROI and nuclei predictions paths from directory_path above
roi_path = directory_path / "ROIs"
nuclei_preds_path = directory_path / "nuclei_preds" / segmentation_type / model_name

# Extract the experiment name from the data directory path
experiment_id = directory_path.name

# Construct the object classifier path
obj_class_path = Path("./APOC_ObjectClassifiers") / experiment_id

# Define output folder for results
results_folder = Path("results") / experiment_id / segmentation_type / model_name

# Create the necessary folder structure if it does not exist
try:
    os.makedirs(str(results_folder))
    print(f"Output folder created: {results_folder}")
except FileExistsError:
    print(f"Output folder already exists: {results_folder}")

# List of subfolder names
try:
    roi_names = [folder.name for folder in roi_path.iterdir() if folder.is_dir()]

except FileNotFoundError:
    roi_names = ["full_image"]
        
print(f"The following regions of interest will be analyzed: {roi_names}")

for image in tqdm(images):

    # Create an empty list to store all stats extracted from each image
    stats = []

    # Read image, apply slicing if needed and return filename and img as a np array
    img, filename = read_image(image, slicing_factor_xy, slicing_factor_z)

    # Generate maximum intensity projection 
    img_mip = maximum_intensity_projection(img)

    for roi_name in tqdm(roi_names):
        print(f"\nAnalyzing ROI: {roi_name}")

        # Read the user defined ROIs, in case of full image analysis generate a label covering the entire image
        try:
            # Read previously defined ROIs
            user_roi = tifffile.imread(roi_path / roi_name / f"{filename}.tiff")

        except FileNotFoundError:
            # Extract the xy dimensions of the input image 
            img_shape = img_mip.shape
            img_xy_dims = img_shape[-2:]

            # Create a label covering the entire image
            user_roi = np.ones(img_xy_dims).astype(np.uint8)

        # Read previously predicted nuclei labels, if not present generate nuclei predictions and save them
        try:
            # Read the nuclei predictions per ROI
            labels = tifffile.imread(nuclei_preds_path / roi_name / f"{filename}.tiff")
            print(f"Pre-computed nuclei labels found for {filename}")
            # Remove labels touching ROI edge (in place for nuclei predictions generated before "remove_labels_touchin_roi_edge" was implemented)
            print("Removing nuclei labels touching ROI edge")
            labels = remove_labels_touching_roi_edge(labels, user_roi)

        except FileNotFoundError:
            print(f"Generating nuclei labels for {filename}")

            # If 3D-segmentation input nuclei_img is a 3D-stack
            if segmentation_type == "3D":
                # Slice the nuclei stack
                nuclei_img = extract_nuclei_stack(img, nuclei_channel)

            # If 2D-segmentation input nuclei_img is a max intensity projection of said 3D-stack
            elif segmentation_type == "2D":
                # Slice the nuclei stack
                nuclei_img = extract_nuclei_stack(img, nuclei_channel)
                nuclei_img = np.max(nuclei_img, axis=0)

            # We will create a mask where roi is greater than or equal to 1
            mask = (user_roi >= 1).astype(np.uint8)

            # 3D segmentation logic, extend 2D mask across the entire stack volume
            if segmentation_type == "3D":
                # Extract the number of z-slices to extend the mask
                slice_nr = img.shape[1]
                # Extend the mask across the entire volume
                mask = np.tile(mask, (slice_nr, 1, 1))
                # Apply the mask to nuclei_img, setting all other pixels to 0
                masked_nuclei_img = np.where(mask, nuclei_img, 0)
            elif segmentation_type == "2D":
                # Apply the mask to nuclei_img, setting all other pixels to 0
                masked_nuclei_img = np.where(mask, nuclei_img, 0)

            # Segment nuclei and return labels
            labels = segment_nuclei(masked_nuclei_img, segmentation_type, model, n_tiles)

            # Remove labels touching ROI edge
            print("Removing nuclei labels touching ROI edge")
            labels = remove_labels_touching_roi_edge(labels, user_roi)

            # Save nuclei labels as .tiff files to reuse them later
            try:
                os.makedirs(nuclei_preds_path / roi_name, exist_ok=True)
            except Exception as e:
                print(f"Error creating directory {nuclei_preds_path / roi_name}: {e}")

            # Construct path to store
            path_to_store = nuclei_preds_path / roi_name / f"{filename}.tiff"
            print(f"Saving nuclei labels to {path_to_store}")
            try:
                tifffile.imwrite(path_to_store, labels)
            except Exception as e:
                print(f"Error saving file {path_to_store}: {e}")

        # Loop through each marker
        for marker in markers:

            # Extract marker_name
            marker_name = marker[0] 

            # Retrieve the first and second values (channel and location) of the corresponding tuple in markers
            for item in markers:
                if item[0] == marker_name:
                    marker_channel = item[1]
                    location = item[2]
                    break  # Stop searching once the marker is found

            if location == "cytoplasm":
                if segmentation_type == "3D":
                    print(f"Generating {segmentation_type} cytoplasm labels for: {marker_name}")
                    # Simulate a cytoplasm by dilating the nuclei and subtracting the nuclei mask afterwards
                    labels = simulate_cytoplasm_chunked_3d(labels, dilation_radius=2, erosion_radius=0, chunk_size=(1, 1024, 1024))

                elif segmentation_type == "2D":
                    print(f"Generating {segmentation_type} cytoplasm labels for: {marker_name}")
                    # Simulate a cytoplasm by dilating the nuclei and subtracting the nuclei mask afterwards
                    labels = simulate_cytoplasm(labels, dilation_radius=2, erosion_radius=0)

            elif location == "cell":
                if segmentation_type == "3D":
                    print(f"Generating {segmentation_type} cell labels for: {marker_name}")
                    # Simulate a cell volume by dilating the nuclei 
                    labels = simulate_cell_chunked_3d(labels, dilation_radius=2, erosion_radius=0, chunk_size=(1, 1024, 1024))

                elif segmentation_type == "2D":
                    print(f"Generating {segmentation_type} cell labels for: {marker_name}")
                    # Simulate a cytoplasm by dilating the nuclei and subtracting the nuclei mask afterwards
                    labels = simulate_cell(labels, dilation_radius=2, erosion_radius=0)

            # Classify labels based on their corresponding object classifier
            cl_filename = f"./{obj_class_path}/ObjClass_{segmentation_type}_ch{marker_channel}.cl"

            # Load the classifier from disc to use the latest version
            classifier = apoc.ObjectClassifier(cl_filename)

            # If 3D-segmentation input marker_img is a 3D-stack
            if segmentation_type == "3D":
                # Slice the img stack
                marker_img = img[marker_channel]

            # If 2D-segmentation input marker_img is a max intensity projection of said 3D-stack
            elif segmentation_type == "2D":
                # Slice the img stack
                marker_img = img_mip[marker_channel]

            # Determine object classification
            print(f"Classifying labels according to {marker_name} intensities...")
            result = classifier.predict(labels, marker_img)

            # Extract unique class values from result and loop through them
            unique_classes = np.unique(result)
            unique_classes = unique_classes[unique_classes != 0]  # Exclude background label

            for label in unique_classes:

                # Retrieve class and transform into a string
                subpopulation = str(label)

                # Create a boolean array (mask) where values match the label (class) in result
                class_mask = cle.pull(result) == label
                class_mask = class_mask.astype(bool) # Convert into boolean to allow for indexing later on

                # Display resulting masks for each class in Napari 
                # viewer.add_labels(class_mask, name=f'{marker_name}{subpopulation}_class')

                # Find nuclei labels that colocalize with said class (mask) using Numpy indexing
                positive_labels = np.unique(labels[class_mask])
                positive_labels = positive_labels[positive_labels != 0]  # Remove background label

                # Extract your information of interest
                total_cells = len(np.unique(labels)) - 1
                marker_pos_cells = len(np.unique(positive_labels)) - 1

                # Calculate "%_marker+_cells" and avoid division by zero errors
                try:
                    perc_marker_pos_cells = (marker_pos_cells * 100) / total_cells
                except ZeroDivisionError:
                    perc_marker_pos_cells = 0

                # Create a dictionary containing all extracted info per masked image
                stats_dict = {
                            "filename": filename,
                            "ROI": roi_name,
                            "population": f'{marker_name}_{subpopulation}',
                            "marker": marker_name,
                            "marker_location":location,
                            "total_cells": total_cells,
                            "marker+_cells": marker_pos_cells,
                            "%_marker+_cells": perc_marker_pos_cells,
                            "nuclei_ch": nuclei_channel,
                            "marker_ch": marker_channel,
                            "slicing_factor_xy": slicing_factor_xy,
                            "slicing_factor_z": slicing_factor_z
                            }

                # Append the current data point to the stats_list
                stats.append(stats_dict)

    # Transform into a dataframe to store it as .csv later
    df = pd.DataFrame(stats)

    # Define the .csv path
    csv_path = results_folder / f"BP_marker_+_label_obj_class.csv"

    # Append to the .csv with new data points each round
    df.to_csv(csv_path, mode="a", index=False, header=not os.path.isfile(csv_path))

# Show the updated .csv 
csv_df = pd.read_csv(csv_path)

csv_df    

Output folder already exists: results\test_data\3D\MEC0.1
The following regions of interest will be analyzed: ['CA', 'DG']


  0%|          | 0/23 [00:00<?, ?it/s]



Image analyzed: HI1_CONTRA_M8_S6_TR1
Original Array shape: (4, 14, 3803, 2891)
Compressed Array shape: (4, 14, 3803, 2891)



Analyzing ROI: CA
Pre-computed nuclei labels found for HI1_CONTRA_M8_S6_TR1
Removing nuclei labels touching ROI edge
Classifying labels according to ki67 intensities...
Generating 3D cell labels for: neun
Classifying labels according to neun intensities...
Generating 3D cytoplasm labels for: calbindin
Classifying labels according to calbindin intensities...



Analyzing ROI: DG
Pre-computed nuclei labels found for HI1_CONTRA_M8_S6_TR1
Removing nuclei labels touching ROI edge
Classifying labels according to ki67 intensities...
Generating 3D cell labels for: neun
Classifying labels according to neun intensities...
Generating 3D cytoplasm labels for: calbindin


...


Analyzing ROI: DG
Pre-computed nuclei labels found for SHAM1_IPSI_M7_S11_TR2
Removing nuclei labels touching ROI edge
Classifying labels according to ki67 intensities...
Generating 3D cell labels for: neun
Classifying labels according to neun intensities...
Generating 3D cytoplasm labels for: calbindin


utils_stardist.py (533): 
Mismatch in label sets! Nuclei labels: 1332, Cytoplasm labels: 1305.
Missing labels in Cytoplasm: [ 349  683  874  920  921  944  962 1077 1166 1168]...
2.03% of cells lost during nuclei subtraction operation.
If this percentage is too high, consider using 'cell' instead of 'cytoplasm' for marker placement. The nuclei may be too densely packed.


Classifying labels according to calbindin intensities...


100%|██████████| 23/23 [23:56<00:00, 62.45s/it]


,filename,ROI,population,marker,marker_location,total_cells,marker+_cells,%_marker+_cells,nuclei_ch,marker_ch,slicing_factor_xy,slicing_factor_z
0,HI1_CONTRA_M8_S6_TR1,CA,ki67_1,ki67,nucleus,1509,1472,97.548045,3,0,NaN,NaN
1,HI1_CONTRA_M8_S6_TR1,CA,ki67_2,ki67,nucleus,1509,35,2.319417,3,0,NaN,NaN
2,HI1_CONTRA_M8_S6_TR1,CA,neun_1,neun,cell,1509,142,9.410205,3,1,NaN,NaN
3,HI1_CONTRA_M8_S6_TR1,CA,neun_2,neun,cell,1509,456,30.218688,3,1,NaN,NaN
4,HI1_CONTRA_M8_S6_TR1,CA,neun_3,neun,cell,1509,908,60.172300,3,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
313,SHAM1_IPSI_M7_S11_TR2,DG,neun_1,neun,cell,1331,41,3.080391,3,1,NaN,NaN
314,SHAM1_IPSI_M7_S11_TR2,DG,neun_2,neun,cell,1331,697,52.366642,3,1,NaN,NaN
315,SHAM1_IPSI_M7_S11_TR2,DG,neun_3,neun,cell,1331,590,44.327573,3,1,NaN,NaN
316,SHAM1_IPSI_M7_S11_TR2,DG,calbindin_1,calbindin,cytoplasm,1304,176,13.496933,3,2,NaN,NaN


In [26]:
import pandas as pd

# Load the CSV file (update the filename as needed)
df = pd.read_csv(csv_path)

# 1. Get all unique population values
unique_populations = sorted(df["population"].unique())

# 2. Create a lookup for marker and marker_location for each population.
pop_info = df.drop_duplicates("population").set_index("population")[["marker", "marker_location"]].to_dict("index")

# 3. Group the data by 'filename' and 'ROI'
group_keys = ["filename", "ROI"]
updated_groups = []

for (filename, roi), group in df.groupby(group_keys):
    # Assume that within each group, these columns are consistent.
    common_values = group.iloc[0][["total_cells", "nuclei_ch", "marker_ch", "slicing_factor_xy", "slicing_factor_z"]].to_dict()
    
    # Identify which populations are missing in this group.
    present_populations = set(group["population"])
    missing_populations = set(unique_populations) - present_populations
    
    # Create new rows for each missing population.
    new_rows = []
    for pop in missing_populations:
        new_row = {
            "filename": filename,
            "ROI": roi,
            "population": pop,
            "marker": pop_info[pop]["marker"],
            "marker_location": pop_info[pop]["marker_location"],
            "total_cells": common_values["total_cells"],
            "marker+_cells": 0,
            "%_marker+_cells": 0,
            "nuclei_ch": common_values["nuclei_ch"],
            "marker_ch": common_values["marker_ch"],
            "slicing_factor_xy": common_values["slicing_factor_xy"],
            "slicing_factor_z": common_values["slicing_factor_z"]
        }
        new_rows.append(new_row)
    
    # Append the new rows to the existing group
    group_updated = pd.concat([group, pd.DataFrame(new_rows)], ignore_index=True)
    updated_groups.append(group_updated)

# 4. Combine all groups back into a single DataFrame.
result = pd.concat(updated_groups, ignore_index=True)

# Sort the DataFrame (for example by filename, ROI, population) for clarity.
result = result.sort_values(by=["filename", "ROI", "population"]).reset_index(drop=True)

# Save the result back to a CSV file if needed
result.to_csv("data_updated.csv", index=False)

print("CSV has been updated and saved as 'data_updated.csv'")

CSV has been updated and saved as 'data_updated.csv'
